<a href="https://colab.research.google.com/github/noambassat/SupremeCourtClassifier/blob/main/LLM_Court.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import torch
import joblib
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, cross_val_score, StratifiedKFold,train_test_split,cross_val_predict
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score,make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
# from imblearn.under_sampling import RandomUnderSampler

from google.colab import drive


In [2]:
# התחברות ל-Google Drive
drive.mount('/content/drive')



Mounted at /content/drive


In [8]:

folder_id = "בית המשפט - קבצים שנבדקו"
directory_path = f"/content/drive/My Drive/{folder_id}"
classifiers_path = "/content/drive/MyDrive/בית המשפט - מסווגים/"

if not os.path.exists(directory_path):
    print(f"Directory {directory_path} does not exist. Please check the folder path.")
else:
    dataframes = []

    for file_name in os.listdir(directory_path):
        if file_name.endswith(".xlsx") or file_name.endswith(".xls"):
            file_path = os.path.join(directory_path, file_name)
            try:
                df = pd.read_excel(file_path)
                dataframes.append(df)
            except Exception as e:
                print(f"Failed to read {file_name}: {e}")

    if dataframes:
        combined_df = pd.concat(dataframes, ignore_index=True)
        print("All Excel files have been concatenated successfully!")

    #     output_file = "/content/drive/My Drive/combined_excel.xlsx"  # עדכן את הנתיב לשמירת הקובץ
    #     combined_df.to_excel(output_file, index=False)
    #     print(f"Combined file saved at: {output_file}")
    # else:
    #     print("No Excel files found or failed to read.")


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


All Excel files have been concatenated successfully!


In [14]:
len(dataframes)

6

In [36]:
df_1 = pd.DataFrame(dataframes[0])
df_1[~(df_1["הערות "].isna() & df_1["הערות קידוד חוזר"].isna() & df_1["טקסט שמצביע על תוצאת בקשה שונה"].isna())]

,level_0,index,גוף המסמך,מספר הליך,שם הליך,שנת פתיחת ההליך,גוף המסמך חתוך,predicted_probability,prediction_original_model,בקשה לרשות ערעור התקבלה?,טקסט שמצביע על תוצאת בקשה שונה,הערעור התקבל?,"רע""פ בקשה אחרת או דלמטה מיוחד",קידוד,קידוד חוזר,הערות,הערות קידוד חוזר
5,5,2982,"['החלטה בתיק רע""פ 3622/15 בבית המשפט העליון רע...","רע""פ 3622/15","רע""פ 3622/15 יעקב סארה נ. מדינת ישראל",2015,נש שעליו הוריתי – מבוטל איפוא. המבקש יתייצב לר...,0.541677,1,נדחה,לנוכח כל האמור לעיל – הבקשה למתן רשות ערעור נדחית,NaN,NaN,דניאל,NaN,NaN,NaN
31,31,3008,"['החלטה בתיק רע""פ 825/15 בבית המשפט העליון רע""...","רע""פ 825/15","רע""פ 825/15 מרדכי קיסוס נ. מדינת ישראל",2015,"""ד סיגל בלום החלטה כמבוקש וכמוסכם המבקש יתייצב...",0.550864,1,אחר,בקשה לעיכוב ביצוע עונש מאסר,אחר,בקשה אחרת,דניאל,NaN,NaN,NaN
67,67,3044,"['החלטה בתיק רע""פ 54/15 בבית המשפט העליון רע""פ...","רע""פ 54/15","רע""פ 54/15",2015,הבקשה לרשות ערעור נדחית בזאת. המבקשים יתייצבו...,0.559125,1,נדחה,"אשר על כן, הבקשה לרשות ערעור נדחית בזאת",NaN,NaN,דניאל,גל,NaN,NaN
73,73,3051,"['פסק דין בתיק רע""פ 2811/15 בבית המשפט העליון ...","רע""פ 2811/15","רע""פ 2811/15",2015,"מש, וזאת במשך חמישה ימים בשבוע, 8.5 שעות עבודה...",0.986913,1,התקבל,NaN,התקבל,NaN,דניאל,גל,הערה של גל - זהו פסק דין משלים,NaN
114,114,3092,"['פסק דין בתיק רע""פ 2195/15 בבית המשפט העליון ...","רע""פ 2195/15","רע""פ 2195/15 אהוד יפרח נ. מדינת ישראל",2015,"ידי כבוד השופטת דבורה עטר תאריך הישיבה: כ""ג ב...",0.901571,1,נדחה,חזרו בהם באי כוח המבקשים מהבקשות למתן רשות ערע...,NaN,NaN,דניאל,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447,1448,17955,"[' בבית המשפט העליון רע""פ 3867/00 בפני: כבודהש...","רע""פ 3867/00","רע""פ 3867/00 עבד אלמעלק תראבינ נ. מדינת ישראל",2000,ון בערעורו שלהמבקש על פסק דינו של בית משפט השל...,0.993145,1,התקבל,NaN,התקבל,NaN,גל,דניאל,״בית המשפט המחוזי יחזור וידון בערעורו שלהמבקש ...,NaN
1448,1449,17980,"[' בבית המשפט העליון רע""פ 889/00 בפני: כבוד המ...","רע""פ 889/00","רע""פ 889/00 אריק תומר נ. מדינת ישראל",2000,פשר יהיה לנקוט בהליכים יעילים נגדתחנות אחרות. ...,0.995475,1,חלקי,״פסק-דין לפי החלטת השופטת שטרסברג-כהן ניתנה למ...,נדחה,NaN,NaN,NaN,NaN,NaN
1453,1453,17994,[' בבית המשפט העליון בשבתו כבית משפט לערעורים ...,"רע""פ 3253/00","רע""פ 3253/00 רנ שנאנ נ. מדינת ישראל",2000,"בשבתו כבית משפט לערעורים פליליים רע""פ 3253/00 ...",0.060109,0,נדחה,NaN,NaN,NaN,גל,דניאל,NaN,׳׳בקשת רשות ערעור נוסחה כדרך בו מנוסח ערעור וא...
1462,1462,18169,"[' בבית המשפט העליון בירושליםרע""פ 1123/00 בפני...","רע""פ 1123/00","רע""פ 1123/00 ציונ אמסלמ נ. מדינת ישראל",2000,"99 שניתן על-ידיכבוד השופטים: צ\' סגל, מ\'שידלו...",0.919334,1,התקבל,NaN,חלקי,NaN,גל,דניאל,NaN,גזר הדין יבוטל והעניין יחזור לבית המשפטהמחוזי ...


In [4]:
combined_df

,level_0,index,גוף המסמך,מספר הליך,שם הליך,שנת פתיחת ההליך,גוף המסמך חתוך,predicted_probability,prediction_original_model,בקשה לרשות ערעור התקבלה?,...,Name of District Court,Month of Supreme Court decision,Justice 2’s activity,District Court ruling favored,Month of Magistrates' Court decision,Supreme Court ruling favored,Legal status of appellant in the Supreme Court,Type of Case,Who appealed the Magistrates’ Court decision (if applicable),Name(s) of Appellee in the Supreme Court
0,0.0,2977.0,"['החלטה בתיק רע""פ 5389/15 בבית המשפט העליון רע...","רע""פ 5389/15","רע""פ 5389/15",2015.0,"ישראל (24.3.1997); בר""ע 2853/91 פלוני נ\' מדינ...",0.016466,0.0,נדחה,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,2978.0,"['החלטה בתיק רע""פ 2487/15 בבית המשפט העליון רע...","רע""פ 2487/15","רע""פ 2487/15 אייל אבולפיה נ. עיריית רמת גנ",2015.0,ופה המוקנית לתובע לצורך מתן מענה לבקשה לביטול ...,0.046868,0.0,נדחה,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,2979.0,"['החלטה בתיק רע""פ 4086/15 בבית המשפט העליון רע...","רע""פ 4086/15","רע""פ 4086/15 עבד אגבאריה נ. מדינת ישראל",2015.0,"וני להכשרת המבנה, הרי שהאינטרס הציבורי של דחיק...",0.029892,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,2980.0,"['פסק דין בתיק רע""פ 8866/15 בבית המשפט העליון ...","רע""פ 8866/15","רע""פ 8866/15 חאזמ רדיע נ. מדינת ישראל",2015.0,"בין רכיב המאסר לרכיב הפסילה, יש מקום לשוב ולד...",0.958560,1.0,התקבל,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,2981.0,"['החלטה בתיק רע""פ 1094/15 בבית המשפט העליון רע...","רע""פ 1094/15","רע""פ 1094/15 שרית יחיאלוב נ. מדינת ישראל",2015.0,. עוד אציע לחברותיי כי עונש המאסר בפועל שהושת ...,0.365762,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tel-Aviv-Jaffa,April,NaN,NaN,July,NaN,Individual,Crim + Sent,Defendant,State of Israel
8988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Be'er Sheva,April,NaN,NaN,cannot tell,NaN,Individual,Crim + Sent,Plaintiff,State of Israel
8989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tel-Aviv-Jaffa,April,NaN,NaN,January,NaN,Individual,Crim + Ev Crim + Sent,Defendant,State of Israel
8990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Tel-Aviv-Jaffa,April,NaN,NaN,cannot tell,NaN,Individual,Crim + Sent,Defendant,State of Israel


In [25]:
data = combined_df[['גוף המסמך', 'מספר הליך', 'שם הליך',
       'שנת פתיחת ההליך', 'גוף המסמך חתוך', 'בקשה לרשות ערעור התקבלה?',
       'טקסט שמצביע על תוצאת בקשה שונה', 'הערעור התקבל?',
       'רע"פ בקשה אחרת או דלמטה מיוחד', 'הערות ',
       'הערות קידוד חוזר',
       'טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)',
       'הערות תוצאת ערעור', 'Outcome of case']]

In [26]:
data.columns

Index(['גוף המסמך', 'מספר הליך', 'שם הליך', 'שנת פתיחת ההליך',
       'גוף המסמך חתוך', 'בקשה לרשות ערעור התקבלה?',
       'טקסט שמצביע על תוצאת בקשה שונה', 'הערעור התקבל?',
       'רע"פ בקשה אחרת או דלמטה מיוחד', 'הערות ', 'הערות קידוד חוזר',
       'טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)',
       'הערות תוצאת ערעור', 'Outcome of case'],
      dtype='object')

In [29]:
data[~data['רע"פ בקשה אחרת או דלמטה מיוחד'].isna()]

,גוף המסמך,מספר הליך,שם הליך,שנת פתיחת ההליך,גוף המסמך חתוך,בקשה לרשות ערעור התקבלה?,טקסט שמצביע על תוצאת בקשה שונה,הערעור התקבל?,"רע""פ בקשה אחרת או דלמטה מיוחד",הערות,הערות קידוד חוזר,טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**),הערות תוצאת ערעור,Outcome of case
31,"['החלטה בתיק רע""פ 825/15 בבית המשפט העליון רע""...","רע""פ 825/15","רע""פ 825/15 מרדכי קיסוס נ. מדינת ישראל",2015.0,"""ד סיגל בלום החלטה כמבוקש וכמוסכם המבקש יתייצב...",אחר,בקשה לעיכוב ביצוע עונש מאסר,אחר,בקשה אחרת,NaN,NaN,NaN,NaN,NaN
132,"['החלטה בתיק רע""פ 7779/15 בבית המשפט העליון רע...","רע""פ 7779/15","רע""פ 7779/15 חברת כצ עבודות צבע בע""מ נ. מדינת ...",2015.0,"על, אשר הושת על המבקש 2, יעוכב עד ליום 5.4.201...",התקבל,בקשה מוסכמת לעיכוב ביצוע עונש מאסר,התקבל,בקשה אחרת,NaN,NaN,NaN,NaN,NaN
177,"['החלטה בתיק רע""פ 1787/15 בבית המשפט העליון רע...","רע""פ 1787/15","רע""פ 1787/15 אדי עמר נ. מדינת ישראל",2015.0,שרון נהרי החלטה עונש המאסר לריצוי בפועל אשר הו...,אחר,בקשה לעיכוב ביצוע גזר דין,NaN,בקשה אחרת,NaN,NaN,NaN,NaN,NaN
189,"['החלטה בתיק רע""פ 3101/15 בבית המשפט העליון רע...","רע""פ 3101/15","רע""פ 3101/15 עמיר אבו רמילה נ. מדינת ישראל",2015.0,"015. המבקש יתייצב, איפוא, במועד הנ""ל, בבית הסו...",אחר,בקשה לדחיית מועד התייצבות למאסר,NaN,בקשה אחרת,NaN,NaN,NaN,NaN,NaN
282,"['החלטה בתיק רע""פ 3572/15 בבית המשפט העליון רע...","רע""פ 3572/15","רע""פ 3572/15 יעקב אלימלכ נ. מדינת ישראל",2015.0,יבה: מדינת ישראל בקשה נוספת לעיכוב ביצוע פסק ד...,אחר,בקשה נוספת לעיכוב ביצוע,NaN,בקשה אחרת,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5460,"['החלטה בתיק רע""פ 5978/13 בבית המשפט העליון רע...","רע""פ 5978/13","רע""פ 5978/13 צבי כהנ ואחיו חברה קבלנית לעבודות...",2013.0,"8/13 בבית המשפט העליון רע""פ 5978/13 - א\' לפני...",NaN,NaN,NaN,בקשה אחרת,בקשה לעיון חוזר בשם המבקשים,NaN,NaN,NaN,NaN
5593,"['החלטה בתיק רע""פ 5155/17 בבית המשפט העליון רע...","רע""פ 5155/17","רע""פ 5155/17",2017.0,ם לעיכוב מועד תחילת ריצוי עונש עבודות שירות בש...,NaN,NaN,NaN,בקשה אחרת,בקשה מוסכמת מטעם הצדדים לעיכוב מועד תחילת ריצו...,NaN,NaN,NaN,NaN
5613,"['החלטה בתיק רע""פ 9952/17 בבית המשפט העליון רע...","רע""פ 9952/17","רע""פ 9952/17 מוחמד אדעיס נ. מדינת ישראל",2017.0,"'החלטה בתיק רע""פ 9952/17 בבית המשפט העליון רע""...",NaN,NaN,NaN,בקשה אחרת,נראה שמדובר בבקשה לדחיית ריצוי העונש,NaN,NaN,NaN,NaN
5635,"['החלטה בתיק רע""פ 6245/17 בבית המשפט העליון רע...","רע""פ 6245/17","רע""פ 6245/17 דב איכנולד נ. יאחב""ל- משטרת ישראל",2017.0,"יאחב""ל - משטרת ישראל תגובה לבקשה לרשות ערר מיו...",NaN,NaN,NaN,בקשה אחרת,תגובה לבקשה לרשות ערר,NaN,NaN,NaN,NaN


In [27]:
check_data = data.dropna(subset = ["הערות ","טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)","טקסט שמצביע על תוצאת בקשה שונה",'הערות תוצאת ערעור'],how="all")

In [28]:
check_data

,גוף המסמך,מספר הליך,שם הליך,שנת פתיחת ההליך,גוף המסמך חתוך,בקשה לרשות ערעור התקבלה?,טקסט שמצביע על תוצאת בקשה שונה,הערעור התקבל?,"רע""פ בקשה אחרת או דלמטה מיוחד",הערות,הערות קידוד חוזר,טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**),הערות תוצאת ערעור,Outcome of case
5,"['החלטה בתיק רע""פ 3622/15 בבית המשפט העליון רע...","רע""פ 3622/15","רע""פ 3622/15 יעקב סארה נ. מדינת ישראל",2015.0,נש שעליו הוריתי – מבוטל איפוא. המבקש יתייצב לר...,נדחה,לנוכח כל האמור לעיל – הבקשה למתן רשות ערעור נדחית,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,"['החלטה בתיק רע""פ 825/15 בבית המשפט העליון רע""...","רע""פ 825/15","רע""פ 825/15 מרדכי קיסוס נ. מדינת ישראל",2015.0,"""ד סיגל בלום החלטה כמבוקש וכמוסכם המבקש יתייצב...",אחר,בקשה לעיכוב ביצוע עונש מאסר,אחר,בקשה אחרת,NaN,NaN,NaN,NaN,NaN
67,"['החלטה בתיק רע""פ 54/15 בבית המשפט העליון רע""פ...","רע""פ 54/15","רע""פ 54/15",2015.0,הבקשה לרשות ערעור נדחית בזאת. המבקשים יתייצבו...,נדחה,"אשר על כן, הבקשה לרשות ערעור נדחית בזאת",NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,"['פסק דין בתיק רע""פ 2811/15 בבית המשפט העליון ...","רע""פ 2811/15","רע""פ 2811/15",2015.0,"מש, וזאת במשך חמישה ימים בשבוע, 8.5 שעות עבודה...",התקבל,NaN,התקבל,NaN,הערה של גל - זהו פסק דין משלים,NaN,NaN,NaN,NaN
114,"['פסק דין בתיק רע""פ 2195/15 בבית המשפט העליון ...","רע""פ 2195/15","רע""פ 2195/15 אהוד יפרח נ. מדינת ישראל",2015.0,"ידי כבוד השופטת דבורה עטר תאריך הישיבה: כ""ג ב...",נדחה,חזרו בהם באי כוח המבקשים מהבקשות למתן רשות ערע...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5460,"['החלטה בתיק רע""פ 5978/13 בבית המשפט העליון רע...","רע""פ 5978/13","רע""פ 5978/13 צבי כהנ ואחיו חברה קבלנית לעבודות...",2013.0,"8/13 בבית המשפט העליון רע""פ 5978/13 - א\' לפני...",NaN,NaN,NaN,בקשה אחרת,בקשה לעיון חוזר בשם המבקשים,NaN,NaN,NaN,NaN
5593,"['החלטה בתיק רע""פ 5155/17 בבית המשפט העליון רע...","רע""פ 5155/17","רע""פ 5155/17",2017.0,ם לעיכוב מועד תחילת ריצוי עונש עבודות שירות בש...,NaN,NaN,NaN,בקשה אחרת,בקשה מוסכמת מטעם הצדדים לעיכוב מועד תחילת ריצו...,NaN,NaN,NaN,NaN
5613,"['החלטה בתיק רע""פ 9952/17 בבית המשפט העליון רע...","רע""פ 9952/17","רע""פ 9952/17 מוחמד אדעיס נ. מדינת ישראל",2017.0,"'החלטה בתיק רע""פ 9952/17 בבית המשפט העליון רע""...",NaN,NaN,NaN,בקשה אחרת,נראה שמדובר בבקשה לדחיית ריצוי העונש,NaN,NaN,NaN,NaN
5635,"['החלטה בתיק רע""פ 6245/17 בבית המשפט העליון רע...","רע""פ 6245/17","רע""פ 6245/17 דב איכנולד נ. יאחב""ל- משטרת ישראל",2017.0,"יאחב""ל - משטרת ישראל תגובה לבקשה לרשות ערר מיו...",NaN,NaN,NaN,בקשה אחרת,תגובה לבקשה לרשות ערר,NaN,NaN,NaN,NaN


In [30]:
check_data[~check_data['רע"פ בקשה אחרת או דלמטה מיוחד'].isna()]

,גוף המסמך,מספר הליך,שם הליך,שנת פתיחת ההליך,גוף המסמך חתוך,בקשה לרשות ערעור התקבלה?,טקסט שמצביע על תוצאת בקשה שונה,הערעור התקבל?,"רע""פ בקשה אחרת או דלמטה מיוחד",הערות,הערות קידוד חוזר,טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**),הערות תוצאת ערעור,Outcome of case
31,"['החלטה בתיק רע""פ 825/15 בבית המשפט העליון רע""...","רע""פ 825/15","רע""פ 825/15 מרדכי קיסוס נ. מדינת ישראל",2015.0,"""ד סיגל בלום החלטה כמבוקש וכמוסכם המבקש יתייצב...",אחר,בקשה לעיכוב ביצוע עונש מאסר,אחר,בקשה אחרת,NaN,NaN,NaN,NaN,NaN
132,"['החלטה בתיק רע""פ 7779/15 בבית המשפט העליון רע...","רע""פ 7779/15","רע""פ 7779/15 חברת כצ עבודות צבע בע""מ נ. מדינת ...",2015.0,"על, אשר הושת על המבקש 2, יעוכב עד ליום 5.4.201...",התקבל,בקשה מוסכמת לעיכוב ביצוע עונש מאסר,התקבל,בקשה אחרת,NaN,NaN,NaN,NaN,NaN
177,"['החלטה בתיק רע""פ 1787/15 בבית המשפט העליון רע...","רע""פ 1787/15","רע""פ 1787/15 אדי עמר נ. מדינת ישראל",2015.0,שרון נהרי החלטה עונש המאסר לריצוי בפועל אשר הו...,אחר,בקשה לעיכוב ביצוע גזר דין,NaN,בקשה אחרת,NaN,NaN,NaN,NaN,NaN
189,"['החלטה בתיק רע""פ 3101/15 בבית המשפט העליון רע...","רע""פ 3101/15","רע""פ 3101/15 עמיר אבו רמילה נ. מדינת ישראל",2015.0,"015. המבקש יתייצב, איפוא, במועד הנ""ל, בבית הסו...",אחר,בקשה לדחיית מועד התייצבות למאסר,NaN,בקשה אחרת,NaN,NaN,NaN,NaN,NaN
282,"['החלטה בתיק רע""פ 3572/15 בבית המשפט העליון רע...","רע""פ 3572/15","רע""פ 3572/15 יעקב אלימלכ נ. מדינת ישראל",2015.0,יבה: מדינת ישראל בקשה נוספת לעיכוב ביצוע פסק ד...,אחר,בקשה נוספת לעיכוב ביצוע,NaN,בקשה אחרת,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5460,"['החלטה בתיק רע""פ 5978/13 בבית המשפט העליון רע...","רע""פ 5978/13","רע""פ 5978/13 צבי כהנ ואחיו חברה קבלנית לעבודות...",2013.0,"8/13 בבית המשפט העליון רע""פ 5978/13 - א\' לפני...",NaN,NaN,NaN,בקשה אחרת,בקשה לעיון חוזר בשם המבקשים,NaN,NaN,NaN,NaN
5593,"['החלטה בתיק רע""פ 5155/17 בבית המשפט העליון רע...","רע""פ 5155/17","רע""פ 5155/17",2017.0,ם לעיכוב מועד תחילת ריצוי עונש עבודות שירות בש...,NaN,NaN,NaN,בקשה אחרת,בקשה מוסכמת מטעם הצדדים לעיכוב מועד תחילת ריצו...,NaN,NaN,NaN,NaN
5613,"['החלטה בתיק רע""פ 9952/17 בבית המשפט העליון רע...","רע""פ 9952/17","רע""פ 9952/17 מוחמד אדעיס נ. מדינת ישראל",2017.0,"'החלטה בתיק רע""פ 9952/17 בבית המשפט העליון רע""...",NaN,NaN,NaN,בקשה אחרת,נראה שמדובר בבקשה לדחיית ריצוי העונש,NaN,NaN,NaN,NaN
5635,"['החלטה בתיק רע""פ 6245/17 בבית המשפט העליון רע...","רע""פ 6245/17","רע""פ 6245/17 דב איכנולד נ. יאחב""ל- משטרת ישראל",2017.0,"יאחב""ל - משטרת ישראל תגובה לבקשה לרשות ערר מיו...",NaN,NaN,NaN,בקשה אחרת,תגובה לבקשה לרשות ערר,NaN,NaN,NaN,NaN


In [ ]:
data[data["גוף המסמך חתוך"].isna()].shape

(0, 70)

In [ ]:
data[data["גוף המסמך חתוך מסווג שני"].isna()].shape

(551, 70)

In [ ]:
data['רע"פ בקשה אחרת או דלמטה מיוחד'].value_counts()


,count
"רע""פ בקשה אחרת או דלמטה מיוחד",
בקשה אחרת,83
דלמטה מיוחד,15


In [ ]:
data[(data['רע"פ בקשה אחרת או דלמטה מיוחד'] == 'בקשה אחרת') | (data['רע"פ בקשה אחרת או דלמטה מיוחד'] == 'דלמטה מיוחד')].shape[0]

98

In [ ]:
data["אחר"] = data.apply(
    lambda row: 1 if row["בקשה לרשות ערעור התקבלה?"] == "אחר" or not pd.isna(row['רע"פ בקשה אחרת או דלמטה מיוחד']) else 0, axis=1
)

<ipython-input-11-188207f6ee3b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["אחר"] = data.apply(


In [ ]:
data["אחר"].value_counts()

,count
אחר,
0,567
1,135


In [ ]:
data.drop_duplicates(subset=["מספר הליך"], inplace=True)

<ipython-input-13-db74405e68a1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop_duplicates(subset=["מספר הליך"], inplace=True)


In [ ]:
check = data[data["אחר"]==1]
check[["בקשה לרשות ערעור התקבלה?",'רע"פ בקשה אחרת או דלמטה מיוחד','גוף המסמך חתוך']].to_excel("check.xlsx")

In [ ]:

# פונקציה לחיתוך הטקסט בהתאם לכללים
def trim_text(text,min_length=400,last_sen=30):


    # שמירת אורך מקורי לאבחון
    original_length = len(text)

    # הסרת מספר תווים מסוף הטקסט
    text = text[:-last_sen]

    # חיתוך ל-min_length האחרונים אם הטקסט ארוך יותר מהמינימום
    if len(text) > min_length:
        text = text[-min_length:]

    # הדפסת פידבק רק אם הטקסט עבר שינוי
    # if len(text) != original_length:
    #     print(f"Trimmed Text (Original Length: {original_length}, Trimmed Length: {len(text)}):")
    #     formatted_text = "\n".join([text[i:i+80] for i in range(0, len(text), 80)])
    #     print(formatted_text)
    #     print("##*******************************************************####")
    return text


In [ ]:
data["גוף המסמך חתוך לתיקים אחרים"] = data["גוף המסמך"].apply(trim_text)


<ipython-input-16-d3497829c694>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["גוף המסמך חתוך לתיקים אחרים"] = data["גוף המסמך"].apply(trim_text)


# Embedding

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dean-ai/sentence_transformer_Legal-heBERT")
model = AutoModel.from_pretrained("dean-ai/sentence_transformer_Legal-heBERT")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings[0]

In [ ]:


data["גוף המסמך חתוך לתיקים אחרים - embeddings"] = data["גוף המסמך חתוך לתיקים אחרים"] .apply(lambda text: get_embeddings(text))


מתחילים לחשב אימבדינגים על גוף המסמך החתוך...
אימבדינגים 2 חושבו בהצלחה!


<ipython-input-19-157ec76b3a8c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["גוף המסמך חתוך לתיקים אחרים - embeddings"] = data["גוף המסמך חתוך לתיקים אחרים"] .apply(lambda text: get_embeddings(text))


In [ ]:
output_file = "/content/drive/My Drive/all_OTHER_cases_with_embeddings.xlsx"
data.to_excel(output_file, index=False)
print(f"Combined file saved at: {output_file}")

Combined file saved at: /content/drive/My Drive/all_OTHER_cases_with_embeddings.xlsx


# Split and train classifier

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.under_sampling import RandomUnderSampler
import joblib

# הכנת הנתונים
X = np.array(data["גוף המסמך חתוך לתיקים אחרים - embeddings"].tolist())  # הפיכת העמודה לרשימה
y = data["אחר"].values

# שמירת אינדקסים מקוריים
data = data.reset_index(drop=True)  # איפוס האינדקס של ה-DataFrame
original_indices = data.index.values  # שמירת האינדקסים המקוריים

# הוספת אינדקסים לנתונים
indices = np.arange(len(X))
X_with_indices = np.hstack([X, indices.reshape(-1, 1)])  # הוספת אינדקסים כעמודה אחרונה

# ביצוע undersampling
undersampler = RandomUnderSampler(sampling_strategy=0.9, random_state=42)
X_resampled_with_indices, y_resampled = undersampler.fit_resample(X_with_indices, y)

# הפרדת האינדקסים מסט האימון
X_resampled = X_resampled_with_indices[:, :-1]  # כל העמודות פרט לעמודת האינדקסים
indices_resampled = X_resampled_with_indices[:, -1].astype(int)  # עמודת האינדקסים

# יצירת סט הבדיקה מהאינדקסים שלא נבחרו
indices_test = np.setdiff1d(indices, indices_resampled)
X_test = X[indices_test]
y_test = y[indices_test]

# הגדרת משקלות לסיווג לא מאוזן
class_weights = {0: 2, 1: 1}

# יצירת המודל של Logistic Regression
classifier = LogisticRegression(class_weight=class_weights, random_state=42, max_iter=1000)

# אימון המודל על ה-TRAIN SET
classifier.fit(X_resampled, y_resampled)

# חיזוי הסתברויות על ה-TRAIN SET
y_train_pred_prob = classifier.predict_proba(X_resampled)[:, 1]
y_train_pred = (y_train_pred_prob >= 0.56).astype(int)

# חיזוי הסתברויות על ה-TEST SET
y_test_pred_prob = classifier.predict_proba(X_test)[:, 1]
y_test_pred = (y_test_pred_prob >= 0.56).astype(int)
# שמירת ההסתברויות והחיזוי בדאטה המקורי עבור כל הנתונים
data["Prediction"] = np.nan
data["Prediction_Probability"] = np.nan

# עדכון סט האימון
data.loc[indices_resampled, "Prediction"] = y_train_pred
data.loc[indices_resampled, "Prediction_Probability"] = y_train_pred_prob

# עדכון סט הבדיקה
data.loc[indices_test, "Prediction"] = y_test_pred
data.loc[indices_test, "Prediction_Probability"] = y_test_pred_prob

# חישוב מטריצות בלבול
conf_matrix_train = confusion_matrix(y_resampled, y_train_pred)
conf_matrix_test = confusion_matrix(y_test, y_test_pred)

# דוח ביצועים מלאים
print("\nTrain Set Results:")
print(f"Confusion Matrix (Train):\n{conf_matrix_train}")
print("\nClassification Report (Train):")
print(classification_report(y_resampled, y_train_pred, target_names=["Class 0", "Class 1"]))

print("\nTest Set Results:")
print(f"Confusion Matrix (Test):\n{conf_matrix_test}")
print("\nClassification Report (Test):")
print(classification_report(y_test, y_test_pred, target_names=["Class 0", "Class 1"], zero_division=1))

# שמירת המודל לקובץ לשימוש עתידי
joblib.dump(classifier, classifiers_path+'other_cases_classifier.pkl')
print("Model saved as 'other_cases_classifier.pkl'")



Train Set Results:
Confusion Matrix (Train):
[[150   0]
 [ 16 119]]

Classification Report (Train):
              precision    recall  f1-score   support

     Class 0       0.90      1.00      0.95       150
     Class 1       1.00      0.88      0.94       135

    accuracy                           0.94       285
   macro avg       0.95      0.94      0.94       285
weighted avg       0.95      0.94      0.94       285


Test Set Results:
Confusion Matrix (Test):
[[374  43]
 [  0   0]]

Classification Report (Test):
              precision    recall  f1-score   support

     Class 0       1.00      0.90      0.95       417
     Class 1       0.00      1.00      0.00         0

    accuracy                           0.90       417
   macro avg       0.50      0.95      0.47       417
weighted avg       1.00      0.90      0.95       417

Model saved as 'other_cases_classifier.pkl'


In [ ]:
def export_model_errors(data, filename="model_errors.xlsx"):
    """
    Exports FP and FN errors to an Excel file with the document body and prediction probabilities.

    Parameters:
    - data (DataFrame): The DataFrame containing model predictions and probabilities.
    - filename (str): The name of the output Excel file.

    Returns:
    - None
    """
    # טבלת False Negatives: הערך בפועל 1 אך סווג כ-0
    fn_table = data[(data["אחר"] == 1) & (data["Prediction"] == 0)][["גוף המסמך", "Prediction_Probability"]]
    fn_table["Error Type"] = "FN"  # סוג הטעות

    # טבלת False Positives: הערך בפועל 0 אך סווג כ-1
    fp_table = data[(data["אחר"] == 0) & (data["Prediction"] == 1)][["גוף המסמך", "Prediction_Probability"]]
    fp_table["Error Type"] = "FP"  # סוג הטעות

    # איחוד הטבלאות
    errors_table = pd.concat([fn_table, fp_table], ignore_index=True)


    output_file = "/content/drive/My Drive/Other_cases_classifiers_errors_only_TP_TN.xlsx"  # עדכן את הנתיב לשמירת הקובץ
    errors_table.to_excel(output_file, index=False)
    print(f"Combined file saved at: {output_file}")

export_model_errors(data, filename="model_errors.xlsx")


Combined file saved at: /content/drive/My Drive/Other_cases_classifiers_errors_only_TP_TN.xlsx


Conclusion:
classifier treshold for predction: 0.56
text cut : 400 - 30
